# 評価例：プッシュ通知要約機能の監視

評価は**タスク指向**で反復的であり、LLM統合の状況を確認し改善するための最良の方法です。

以下の評価では、**プロンプト変更による回帰の検出**というタスクに焦点を当てます。

私たちのユースケースは以下の通りです：
1. 本番環境のチャット補完リクエストで`store=True`を設定することで、チャット補完リクエストをログに記録しています。なお、管理パネル（https://platform.openai.com/settings/organization/data-controls/data-retention）で「デフォルトでオン」のログ記録を有効にすることも可能です。
2. プロンプトの変更が回帰を引き起こしていないかを確認したいと考えています。

## 評価の構造

評価には「Eval」と「Run」の2つの部分があります。「Eval」はテスト基準の設定と「Run」のデータ構造を保持します。1つのEvalは複数のRunを持つことができ、それぞれがテスト基準を使用して評価されます。

In [1]:
from openai import AsyncOpenAI
import os
import asyncio

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "your-api-key")
client = AsyncOpenAI()

## ユースケース

以下の統合をテストしています。プッシュ通知サマリー機能で、複数のプッシュ通知を受け取り、それらを1つにまとめるものです。これはchat completions呼び出しです。

# テストデータの生成

本番環境のチャット補完リクエストをシミュレートして、2つの異なるプロンプトバージョンをテストし、それぞれのパフォーマンスを確認します。1つ目は「良い」プロンプト、2つ目は「悪い」プロンプトです。これらには異なるメタデータが含まれており、後で使用します。

In [2]:
push_notification_data = [
        """
- New message from Sarah: "Can you call me later?"
- Your package has been delivered!
- Flash sale: 20% off electronics for the next 2 hours!
""",
        """
- Weather alert: Thunderstorm expected in your area.
- Reminder: Doctor's appointment at 3 PM.
- John liked your photo on Instagram.
""",
        """
- Breaking News: Local elections results are in.
- Your daily workout summary is ready.
- Check out your weekly screen time report.
""",
        """
- Your ride is arriving in 2 minutes.
- Grocery order has been shipped.
- Don't miss the season finale of your favorite show tonight!
""",
        """
- Event reminder: Concert starts at 7 PM.
- Your favorite team just scored!
- Flashback: Memories from 3 years ago.
""",
        """
- Low battery alert: Charge your device.
- Your friend Mike is nearby.
- New episode of "The Tech Hour" podcast is live!
""",
        """
- System update available.
- Monthly billing statement is ready.
- Your next meeting starts in 15 minutes.
""",
        """
- Alert: Unauthorized login attempt detected.
- New comment on your blog post: "Great insights!"
- Tonight's dinner recipe: Pasta Primavera.
""",
        """
- Special offer: Free coffee with any breakfast order.
- Your flight has been delayed by 30 minutes.
- New movie release: "Adventures Beyond" now streaming.
""",
        """
- Traffic alert: Accident reported on Main Street.
- Package out for delivery: Expected by 5 PM.
- New friend suggestion: Connect with Emma.
"""]

In [ ]:
PROMPTS = [
    (
        """
        You are a helpful assistant that summarizes push notifications.
        You are given a list of push notifications and you need to collapse them into a single one.
        Output only the final summary, nothing else.
        """,
        "v1"
    ),
    (
        """
        You are a helpful assistant that summarizes push notifications.
        You are given a list of push notifications and you need to collapse them into a single one.
        The summary should be longer than it needs to be and include more information than is necessary.
        Output only the final summary, nothing else.
        """,
        "v2"
    )
]

tasks = []
for notifications in push_notification_data:
    for (prompt, version) in PROMPTS:
        tasks.append(client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "developer", "content": prompt},
                {"role": "user", "content": notifications},
            ],
            store=True,
            metadata={"prompt_version": version, "usecase": "push_notifications_summarizer"},
        ))
await asyncio.gather(*tasks)

作成した補完は https://platform.openai.com/logs で確認できます。

**次のステップに必要なため、チャット補完が表示されていることを確認してください。**

In [ ]:
completions = await client.chat.completions.list()
assert completions.data, "No completions found. You may need to enable logs in your admin panel."
completions.data[0]

# Evalの設定

Evalは複数の*Run*間で共有される設定を保持し、2つのコンポーネントで構成されています：
1. データソース設定 `data_source_config` - 将来の*Run*が準拠するスキーマ（列）。
    - `data_source_config`はJSON Schemaを使用して、Evalで利用可能な変数を定義します。
2. テスト基準 `testing_criteria` - データソースの各*行*に対してインテグレーションが機能しているかどうかを判断する方法。

このユースケースでは、stored-completionsを使用するため、そのdata_source_configを設定します。

**重要**
stored completionsのユースケースは多数存在する可能性が高いため、メタデータを使用してこれらを追跡し、evalを集中的かつタスク指向に保つことが最良の方法です。

In [5]:
# We want our input data to be available in our variables, so we set the item_schema to
# PushNotifications.model_json_schema()
data_source_config = {
    "type": "stored_completions",
    "metadata": {
        "usecase": "push_notifications_summarizer"
    }
}

このdata_source_configは、eval全体で利用可能な変数を定義します。

stored completions configは、eval全体で使用できる2つの変数を提供します：
1. {{item.input}} - completions呼び出しに送信されたメッセージ
2. {{sample.output_text}} - アシスタントからのテキスト応答

**次に、これらの変数を使用してテスト基準を設定します。**

In [6]:
GRADER_DEVELOPER_PROMPT = """
Label the following push notification summary as either correct or incorrect.
The push notification and the summary will be provided below.
A good push notificiation summary is concise and snappy.
If it is good, then label it as correct, if not, then incorrect.
"""
GRADER_TEMPLATE_PROMPT = """
Push notifications: {{item.input}}
Summary: {{sample.output_text}}
"""
push_notification_grader = {
    "name": "Push Notification Summary Grader",
    "type": "label_model",
    "model": "o3-mini",
    "input": [
        {
            "role": "developer",
            "content": GRADER_DEVELOPER_PROMPT,
        },
        {
            "role": "user",
            "content": GRADER_TEMPLATE_PROMPT,
        },
    ],
    "passing_labels": ["correct"],
    "labels": ["correct", "incorrect"],
}

`push_notification_grader`は、モデルグレーダー（llm-as-a-judge）であり、入力`{{item.input}}`と生成された要約`{{sample.output_text}}`を確認して、「正しい」または「間違っている」とラベル付けします。

注意：内部的には、構造化出力を使用しているため、ラベルは常に有効です。

**それでは評価を作成し、データの追加を開始しましょう！**

In [7]:
eval_create_result = await client.evals.create(
    name="Push Notification Completion Monitoring",
    metadata={"description": "This eval monitors completions"},
    data_source_config=data_source_config,
    testing_criteria=[push_notification_grader],
)

eval_id = eval_create_result.id

# 実行の作成

テストクライテリアを使用してevalのセットアップが完了したので、実行を追加し始めることができます。
2つの**プロンプトバージョン**間のパフォーマンスを比較したいと思います。

これを行うために、各プロンプトバージョンに対してメタデータフィルターを使用して、ソースを"stored_completions"として定義するだけです。

In [ ]:
# Grade prompt_version=v1
eval_run_result = await client.evals.runs.create(
    eval_id=eval_id,
    name="v1-run",
    data_source={
        "type": "completions",
        "source": {
            "type": "stored_completions",
            "metadata": {
                "prompt_version": "v1",
            }
        }
    }
)
print(eval_run_result.report_url)

In [ ]:
# Grade prompt_version=v2
eval_run_result_v2 = await client.evals.runs.create(
    eval_id=eval_id,
    name="v2-run",
    data_source={
        "type": "completions",
        "source": {
            "type": "stored_completions",
            "metadata": {
                "prompt_version": "v2",
            }
        }
    }
)
print(eval_run_result_v2.report_url)

念のため、4o-miniの代わりに4oを使って、両方のプロンプトバージョンを出発点として、このプロンプトがどのような性能を示すかを確認してみましょう。

必要なのは、入力メッセージ（{{item.input}}）を参照し、モデルを4oに設定することだけです。4o用の保存された補完結果がまだないため、この評価実行では新しい補完結果が生成されます。

In [ ]:
tasks = []
for prompt_version in ["v1", "v2"]:
    tasks.append(client.evals.runs.create(
        eval_id=eval_id,
        name=f"post-fix-new-model-run-{prompt_version}",
        data_source={
            "type": "completions",
            "input_messages": {
                "type": "item_reference",
                "item_reference": "item.input",
            },
            "model": "gpt-4o",
            "source": {
                "type": "stored_completions",
                "metadata": {
                    "prompt_version": prompt_version,
                }
            }
        },
    ))
result = await asyncio.gather(*tasks)
for run in result:
    print(run.report_url)

そのレポートを確認すると、`prompt_version=v2`にリグレッションがあることが分かります！

## おめでとうございます。バグを発見しました。これを元に戻すか、別のプロンプト変更を行うなどができます！